显示逐时效的产品


In [45]:
import pandas as pd
from nwpc_message_tool import nmc_monitor

engine = nmc_monitor

start_time = (pd.to_datetime("2020-03-01T00:00:00"), pd.to_datetime("2020-03-17T23:00:00"))

system = "nwpc_grapes_gfs"
system = engine.fix_system_name(system)

elastic_server = ""

In [46]:
from nwpc_message_tool.storage import EsMessageStorage

client = EsMessageStorage(
    hosts=elastic_server,
    engine=engine,
)

results = client.get_production_messages(
    system=system,
    production_stream="oper",
    production_type="grib2",
    production_name="orig",
    start_time=start_time
)

results = list(results)

2020-03-18 22:33:58.279 | INFO     | nwpc_message_tool.storage:_get_result:74 - searching from 0 with size 20...
2020-03-18 22:33:58.280 | DEBUG    | nwpc_message_tool.storage:_get_result:80 - search body: {'size': 20, 'from': 0, 'query': {'bool': {'filter': [{'term': {'source': 'nwpc_grapes_gfs'}}, {'range': {'startTime': {'gte': '2020-03-01T00:00:00', 'lte': '2020-03-17T23:00:00'}}}]}}}
2020-03-18 22:33:58.296 | INFO     | nwpc_message_tool.storage:get_production_messages:67 - total: 2856
2020-03-18 22:33:58.297 | INFO     | nwpc_message_tool.storage:get_production_messages:69 - result count: 20
2020-03-18 22:33:58.299 | INFO     | nwpc_message_tool.storage:_get_result:74 - searching from 20 with size 20...
2020-03-18 22:33:58.300 | DEBUG    | nwpc_message_tool.storage:_get_result:80 - search body: {'size': 20, 'from': 20, 'query': {'bool': {'filter': [{'term': {'source': 'nwpc_grapes_gfs'}}, {'range': {'startTime': {'gte': '2020-03-01T00:00:00', 'lte': '2020-03-17T23:00:00'}}}]}}}
2

In [47]:
from nwpc_message_tool._util import get_hour
import numpy as np

df = pd.DataFrame(columns=["start_time", "forecast_hour", "time"])
for result in results:
    hours = get_hour(result)
    message_time = result.time.ceil("S")
    current_df = pd.DataFrame(
        {
            "start_time": [pd.to_datetime(result.start_time)],
            "forecast_hour": [hours],
            "time": [message_time]
        },
        columns=["start_time", "forecast_hour", "time"],
        index=[f"{result.start_time.strftime('%Y%m%d%H')}+{hours:03}"]
    )
    df = df.append(current_df)
df.sort_index()
df

,start_time,forecast_hour,time
2020030400+000,2020-03-04 00:00:00+00:00,0,2020-03-04 12:37:13+08:00
2020030400+003,2020-03-04 00:00:00+00:00,3,2020-03-04 12:38:31+08:00
2020030400+006,2020-03-04 00:00:00+00:00,6,2020-03-04 12:38:54+08:00
2020030400+009,2020-03-04 00:00:00+00:00,9,2020-03-04 12:39:02+08:00
2020030400+012,2020-03-04 00:00:00+00:00,12,2020-03-04 12:39:43+08:00
...,...,...,...
2020031718+027,2020-03-17 18:00:00+00:00,27,2020-03-18 06:15:41+08:00
2020031718+030,2020-03-17 18:00:00+00:00,30,2020-03-18 06:15:52+08:00
2020031718+033,2020-03-17 18:00:00+00:00,33,2020-03-18 06:16:18+08:00
2020031718+036,2020-03-17 18:00:00+00:00,36,2020-03-18 06:16:44+08:00


In [48]:
df["time"] = pd.to_datetime(df["time"], utc=True)
df["start_time"] = pd.to_datetime(df["start_time"], utc=True)
df["forecast_hour"] = pd.to_numeric(df["forecast_hour"])

df["fh"] = df.forecast_hour.astype(str)
df["st"] = df.start_time.apply(lambda x: x.strftime("%Y%m%d%H"))
df["clock"] = df.time - df.start_time
df["clocks"] = df['clock'].astype(np.int64) / int(1e6)
df["time_string"] = df["time"].apply(lambda x: x.strftime("%H:%M:%S"))
df

,start_time,forecast_hour,time,fh,st,clock,clocks,time_string
2020030400+000,2020-03-04 00:00:00+00:00,0,2020-03-04 04:37:13+00:00,0,2020030400,04:37:13,16633000.0,04:37:13
2020030400+003,2020-03-04 00:00:00+00:00,3,2020-03-04 04:38:31+00:00,3,2020030400,04:38:31,16711000.0,04:38:31
2020030400+006,2020-03-04 00:00:00+00:00,6,2020-03-04 04:38:54+00:00,6,2020030400,04:38:54,16734000.0,04:38:54
2020030400+009,2020-03-04 00:00:00+00:00,9,2020-03-04 04:39:02+00:00,9,2020030400,04:39:02,16742000.0,04:39:02
2020030400+012,2020-03-04 00:00:00+00:00,12,2020-03-04 04:39:43+00:00,12,2020030400,04:39:43,16783000.0,04:39:43
...,...,...,...,...,...,...,...,...
2020031718+027,2020-03-17 18:00:00+00:00,27,2020-03-17 22:15:41+00:00,27,2020031718,04:15:41,15341000.0,22:15:41
2020031718+030,2020-03-17 18:00:00+00:00,30,2020-03-17 22:15:52+00:00,30,2020031718,04:15:52,15352000.0,22:15:52
2020031718+033,2020-03-17 18:00:00+00:00,33,2020-03-17 22:16:18+00:00,33,2020031718,04:16:18,15378000.0,22:16:18
2020031718+036,2020-03-17 18:00:00+00:00,36,2020-03-17 22:16:44+00:00,36,2020031718,04:16:44,15404000.0,22:16:44


In [49]:
def check_hour(x):
    if x.hour != 0:
        return None
    else:
        return 1
df["start_time"] = df["start_time"].apply(check_hour)
df = df.dropna(how="any")
df

,start_time,forecast_hour,time,fh,st,clock,clocks,time_string
2020030400+000,1.0,0,2020-03-04 04:37:13+00:00,0,2020030400,04:37:13,16633000.0,04:37:13
2020030400+003,1.0,3,2020-03-04 04:38:31+00:00,3,2020030400,04:38:31,16711000.0,04:38:31
2020030400+006,1.0,6,2020-03-04 04:38:54+00:00,6,2020030400,04:38:54,16734000.0,04:38:54
2020030400+009,1.0,9,2020-03-04 04:39:02+00:00,9,2020030400,04:39:02,16742000.0,04:39:02
2020030400+012,1.0,12,2020-03-04 04:39:43+00:00,12,2020030400,04:39:43,16783000.0,04:39:43
...,...,...,...,...,...,...,...,...
2020031700+216,1.0,216,2020-03-17 04:50:03+00:00,216,2020031700,04:50:03,17403000.0,04:50:03
2020031700+222,1.0,222,2020-03-17 04:50:10+00:00,222,2020031700,04:50:10,17410000.0,04:50:10
2020031700+228,1.0,228,2020-03-17 04:51:12+00:00,228,2020031700,04:51:12,17472000.0,04:51:12
2020031700+234,1.0,234,2020-03-17 04:51:28+00:00,234,2020031700,04:51:28,17488000.0,04:51:28


In [52]:
from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.models import LinearColorMapper, HoverTool

# output_file("p.html")
output_notebook()

Loading BokehJS ...

In [54]:
forecast_hours = df.fh.unique()
start_times = df.st.unique()

cmap = LinearColorMapper(
    palette="Viridis256", 
    low = min(df['clocks']), 
    high = max(df['clocks']),
)

p = figure(
    title="Production", 
    plot_width=30*len(forecast_hours), 
    plot_height=35*len(start_times),
    x_range=forecast_hours, 
    y_range=start_times,
)

r = p.rect(
    "fh", "st", 0.9, 0.9, 
    source=df, 
    fill_alpha=0.6,
    fill_color={
        "field": "clocks",
        "transform": cmap,
    },
)

p.add_tools(HoverTool(
    tooltips=[
        ("time", "@time_string")
    ],
))

show(p)